# Fisheye camera calibration for OpenVSLAM

In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import time
import sys
import os
from os.path import join
import yaml
import glob
from functools import partial

from enum import Enum
class Camera(Enum):
    OpenCV = 1 # OpenCV
    PySpin = 2 # Spinnaker
    
class Pattern(Enum):
    Chessboard = 1
    Circle = 2

%matplotlib inline

# Capture checkboard images
Checkborad pdf file is available at 
 - https://raw.githubusercontent.com/opencv/opencv/master/doc/pattern.png
 - https://raw.githubusercontent.com/opencv/opencv/master/doc/acircles_pattern.png

In [ ]:
# User setting

# Input camera
# cam_kind = Camera.OpenCV
cam_kind = Camera.PySpin

# Pattern
pattern_kind = Pattern.Chessboard
# pattern_kind = Pattern.Circle


In [ ]:
cam_id = 0

if cam_kind == Camera.PySpin:
    sys.path.insert(0, './SpinnakerVideoCapture/python')
    from PySpinCap import PySpinManager
    # start manager
    manager = PySpinManager()
    cap = manager.get_camera(cam_id)
if cam_kind == Camera.OpenCV:
    cap = cv2.VideoCapture(cam_id)
    fps = cap.get(cv2.CAP_PROP_FPS)
    print('Framerate:',fps)


In [ ]:
print('#####################')
print('### start capturing')
elps = []
save_imgs = []
cv2.namedWindow('show img', cv2.WINDOW_KEEPRATIO)
while True:
    start = time.time()

    ret, img = cap.read()
    cv2.imshow('show img', img)
#     show = cv2.resize(img, None, fx=0.5, fy=0.5)
#     cv2.imshow('show img', show)
    key = cv2.waitKey(20)
    elps.append((time.time() - start))

    if key == ord('s'):
        cv2.waitKey(300)
        save_imgs.append(img)
        print('save image:', len(save_imgs))
    elif key == 27:
        break

    if len(elps) == 100:
        print('- FPS:{}'.format(len(elps) / sum(elps)))
        elps = []

cv2.destroyAllWindows()
print('### finish capturing')
print('#####################')


In [ ]:
# Release everything
cap.release()
if cam_kind == Camera.PySpin:
    manager.release()

In [ ]:
save_img_folder = './img{}'.format(time.strftime("%Y%m%d-%H%M"))
try:
    os.mkdir(save_img_folder)
except OSError as exc:
    print(exc)
print(save_img_folder)

In [ ]:
# Save images
for i,it in enumerate(save_imgs):
    print('save image : {:03}'.format(i))
    cv2.imwrite(join(save_img_folder, '{:03}.png'.format(i)), it)

# Calib image

## Select pattern type first

In [ ]:
# load_folder = './img20191027-2136/'
# load_folder = './img20191027-2312/'
load_folder = save_img_folder
print('load folder:', load_folder)

In [ ]:
import glob
import numpy as np
import cv2
import matplotlib.pyplot as plt
import time
%matplotlib inline

images = []
for fname in sorted(glob.glob('{}/*.png'.format(load_folder))):
    img = cv2.imread(fname)
    images.append(img)
    print(fname)

In [ ]:
subpix_criteria = (cv2.TERM_CRITERIA_EPS+cv2.TERM_CRITERIA_MAX_ITER, 30, 0.1)
calibration_flags = cv2.fisheye.CALIB_RECOMPUTE_EXTRINSIC+cv2.fisheye.CALIB_CHECK_COND+cv2.fisheye.CALIB_FIX_SKEW
imgsize = None
if pattern_kind == Pattern.Chessboard:
    pattern_size = (6,9)
    objp = np.zeros((1, pattern_size[0]*pattern_size[1], 3), np.float32)
    objp[0,:,:2] = np.mgrid[0:pattern_size[0], 0:pattern_size[1]].T.reshape(-1, 2)
    detect_flag = cv2.CALIB_CB_ADAPTIVE_THRESH+cv2.CALIB_CB_NORMALIZE_IMAGE # cv2.CALIB_CB_FAST_CHECK+
    findCorners = partial(cv2.findChessboardCorners, patternSize=pattern_size, flags=detect_flag)
    
elif pattern_kind == Pattern.Circle:
    pattern_size = (4, 11)
    objp = []
    for i in range(pattern_size[1]):
        for j in range(pattern_size[0]):
            objp.append([2*j+i%2, i ,0])
    objp = np.array(objp, np.float32)[np.newaxis]
    detect_flag = cv2.CALIB_CB_ASYMMETRIC_GRID+cv2.CALIB_CB_CLUSTERING
    findCorners = partial(cv2.findCirclesGrid, patternSize=pattern_size, flags=detect_flag)


In [ ]:
# check first image
if True:
    img = images[2].copy()
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    # Find the chess board corners
    ret, corners = findCorners(gray)

    if ret:
        cv2.cornerSubPix(gray,corners,(3,3),(-1,-1),subpix_criteria)
        cv2.drawChessboardCorners(img, pattern_size, corners, ret) 
        x_max, y_max = np.max(corners, axis=0)[0]
        x_min, y_min = np.min(corners, axis=0)[0]
        offset = 50
        x_max = int(x_max)+offset
        x_min = int(x_min)-offset
        y_max = int(y_max)+offset
        y_min = int(y_min)-offset
        fig, ax = plt.subplots(1, 2, figsize=(15,8))
        ax[0].imshow(img[y_min:y_max,x_min:x_max,::-1])
        ax[1].imshow(img[:,:,::-1])
    else:
        print("couldn't find corner")

In [ ]:
save_cornerimg = True

objpoints = [] # 3d point in real world space
imgpoints = [] # 2d points in image plane.
for i, img in enumerate(images):
    if imgsize == None:
        imgsize = img.shape[:2]
    else:
        assert imgsize == img.shape[:2], "All images must share the same size."
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    # Find the chess board corners
    ret, corners = findCorners(gray)
    # If found, add object points, image points (after refining them)
    if ret == True:
        print('found!{}: imgpoints id:{}'.format(i, len(objpoints)))
        objpoints.append(objp)
        cv2.cornerSubPix(gray,corners,(3,3),(-1,-1),subpix_criteria)
        imgpoints.append(corners)
        if save_cornerimg:
            show = img.copy()
            cv2.drawChessboardCorners(show, pattern_size, corners, ret) 
            fname = join(load_folder, 'corner{}.jpg'.format(len(imgpoints)-1))
            cv2.imwrite(fname, show)
    else:
        print("couldn't find corner")

In [ ]:
def fisheyeCalib(objpoints, imgpoints, imgsize, reject=[]):
    print("#### FISHEYE CALIB START #####")
    N_OK = len(imgpoints)
    print("Found " + str(N_OK) + " valid images for calibration")
    K = np.zeros((3, 3))
    D = np.zeros((4, 1))
    rvecs = [np.zeros((1, 1, 3), dtype=np.float64) for i in range(N_OK)]
    tvecs = [np.zeros((1, 1, 3), dtype=np.float64) for i in range(N_OK)]

    calibration_flags = cv2.fisheye.CALIB_RECOMPUTE_EXTRINSIC
    calibration_flags |= cv2.fisheye.CALIB_CHECK_COND
    calibration_flags |= cv2.fisheye.CALIB_FIX_SKEW

    #####################
    ## find bad images
    while True:
        try:
            tmp_imgpoints = [imgpoints[i] for i in range(len(imgpoints)) if not i in reject]
            tmp_objpoints = [objpoints[i] for i in range(len(imgpoints)) if not i in reject]
            rms, _, _, _, _ = \
                cv2.fisheye.calibrate(
                    tmp_objpoints, tmp_imgpoints, imgsize[::-1], K, D, rvecs, tvecs,
                    calibration_flags, (cv2.TERM_CRITERIA_EPS+cv2.TERM_CRITERIA_MAX_ITER, 30, 1e-6)
                )
            print('Current rms:', rms)
            break
        except Exception as e:
#             print("type error: " + str(e))
            msg = str(e).split('input array ')[1]
            reject_idx = int(msg.split()[0])
            reject.append(reject_idx+len(reject))
            print('reject_idx:', reject[-1])

    ######################
    ## final refinement
    calibration_flags = cv2.fisheye.CALIB_RECOMPUTE_EXTRINSIC
    calibration_flags |= cv2.fisheye.CALIB_CHECK_COND
    calibration_flags |= cv2.fisheye.CALIB_FIX_SKEW
    calibration_flags |= cv2.fisheye.CALIB_USE_INTRINSIC_GUESS
    
    # use rejected point or not
    if True:
        tmp_imgpoints = [imgpoints[i] for i in range(len(imgpoints)) if not i in reject]
        tmp_objpoints = [objpoints[i] for i in range(len(imgpoints)) if not i in reject]
    else:
        tmp_imgpoints = imgpoints
        tmp_objpoints = objpoints

    rms, K, D, rvecs, tvecs = \
        cv2.fisheye.calibrate(
            tmp_objpoints, tmp_imgpoints, imgsize[::-1], K, D, rvecs, tvecs,
            calibration_flags, (cv2.TERM_CRITERIA_EPS+cv2.TERM_CRITERIA_MAX_ITER, 30, 1e-6)
        )

    print('final result')
    print("K=np.array(" + str(K.tolist()) + ")")
    print("D=np.array(" + str(D.tolist()) + ")")
    print("RMS:", rms)
    print("#### FISHEYE CALIB END #####")
    return K, D, rvecs, tvecs, rms

In [ ]:
print('calib:')
K1, D1, rvecs, tvecs, rms = fisheyeCalib(objpoints, imgpoints, imgsize)

In [ ]:
def printForOpenVSLAM(K, D, img, fps=0, load_folder=load_folder):
    data = {}
    data['fx'] = K[0][0]
    data['fy'] = K[1][1]
    data['cx'] = K[0][2]
    data['cy'] = K[1][2]

    data['k1'] = D[0][0]
    data['k2'] = D[1][0]
    data['k3'] = D[2][0]
    data['k4'] = D[3][0]
    
    data['fps'] = fps
    data['cols'] = img.shape[1]
    data['rows'] = img.shape[0]
    
    print_array=[]
    for it in ['fx', 'fy', 'cx', 'cy']:
        print_array.append('Camera.{}: {}'.format(it, data[it]))
    print_array.append('')
    for it in ['k1', 'k2', 'k3', 'k4']:
        print_array.append('Camera.{}: {}'.format(it, data[it]))
    print_array.append('')
    for it in ['fps', 'cols', 'rows']:
        print_array.append('Camera.{}: {}'.format(it, data[it]))

    with open(join(load_folder,'calib_fisheye_results.txt'), mode='w') as f:
        for it in print_array:
            print(it)
            print(it, file=f)

In [ ]:
printForOpenVSLAM(K, D, img, fps=30, load_folder=load_folder)

# Rectify

In [ ]:
# Load calib
if not 'K' in locals(): 
    load_folder = './img20191028-1026/'
    print('Need to load calibration data:')
    print('Loading from ', load_folder)
    with open(join(load_folder, 'calib_fisheye_results.txt')) as f:
        data = yaml.load(f, Loader=yaml.FullLoader)
        
    K = np.eye(3)
    K[0, 0] = data['Camera.fx']
    K[1, 1] = data['Camera.fy']  
    K[0, 2] = data['Camera.cx']   
    K[1, 2] = data['Camera.cy']   
    D = np.zeros((4, 1))
    D[0, 0] = data['Camera.k1']
    D[1, 0] = data['Camera.k2']
    D[2, 0] = data['Camera.k3']
    D[3, 0] = data['Camera.k4']
    print("K=np.array(" + str(K.tolist()) + ")")
    print("D=np.array(" + str(D.tolist()) + ")")
    
    images = []
    for fname in sorted(glob.glob('{}/*.png'.format(load_folder))):
        img = cv2.imread(fname)
        images.append(img)

In [ ]:
img = images[0]
plt.imshow(img)

## Normal rectification

In [ ]:
newK = K.copy()
newK[0][0]*=2
newK[1][1]*=2
newK

In [ ]:
img_size = (img.shape[1], img.shape[0])
map1, map2 = cv2.fisheye.initUndistortRectifyMap(K, D, np.eye(3), newK, img_size, cv2.CV_16SC2)  # Pass k in 1st parameter, nk in 4th parameter
rectify = cv2.remap(img, map1, map2, interpolation=cv2.INTER_LINEAR, borderMode=cv2.BORDER_CONSTANT)
plt.figure(figsize=(8,8))
plt.imshow(rectify[:,:,::-1])

## Change image size

In [ ]:
new_size=(640, 480)
newK = cv2.fisheye.estimateNewCameraMatrixForUndistortRectify(K, D, img_size, np.eye(3), new_size=new_size)

# Adjust fov
factor = 45/90*newK[0][0]/newK[0][2]
newK[0][0]/= factor
newK[1][1]/= factor
newK

In [ ]:
img_size = (img.shape[1], img.shape[0])
map1, map2 = cv2.fisheye.initUndistortRectifyMap(K, D, np.eye(3), newK, new_size, cv2.CV_16SC2)  # Pass k in 1st parameter, nk in 4th parameter
rectify = cv2.remap(img, map1, map2, interpolation=cv2.INTER_LINEAR, borderMode=cv2.BORDER_CONSTANT)
plt.figure(figsize=(8,8))
plt.imshow(rectify[:,:,::-1])

# Rectify images
## Define map first
e.g.,  
map1, map2 = cv2.fisheye.initUndistortRectifyMap(K, D, np.eye(3), newK, img_size, cv2.CV_16SC2) 

In [ ]:
# Input camera
# cam_kind = Camera.cv2
cam_kind = Camera.PySpin

cam_id = 0

if cam_kind == Camera.PySpin:
    sys.path.insert(0, './SpinnakerVideoCapture/python')
    from PySpinCap import PySpinManager
    # start manager
    manager = PySpinManager()
    cap = manager.get_camera(cam_id)
if cam_kind == Camera.OpenCV:
    cap = cv2.VideoCapture('../datasets/nu_eng2_corridor_3/video.mp4')
    fps = cap.get(cv2.CAP_PROP_FPS)
    print('Framerate:',fps)


In [ ]:
print('#####################')
print('### start capturing')
remap = partial(cv2.remap, interpolation=cv2.INTER_LINEAR, borderMode=cv2.BORDER_CONSTANT)

val_r = 0
val_b = 0

def on_trackbar_red(val):
    global val_r, val_b
    val_r = 2*val/100 + 0.5
    cap.set_white_balance_ratio(val_r, select='Red')
    print('Set white balance {}, {}:'.format(val_r, val_b))
def on_trackbar_blue(val):
    global val_r, val_b
    val_b = 2*val/100 + 0.5
    cap.set_white_balance_ratio(val_b, select='Blue')
    print('Set white balance {}, {}:'.format(val_r, val_b))
    

elps = []
save_imgs = []
winname = 'show img'
cv2.namedWindow(winname, cv2.WINDOW_KEEPRATIO)
if cam_kind == Camera.PySpin:
    cv2.createTrackbar('white_balance_red', winname , 34, 100, on_trackbar_red)
    on_trackbar_red(34)
    cv2.createTrackbar('white_balance_blue', winname , 48, 100, on_trackbar_blue)
    on_trackbar_blue(48)
    
while True:
    start = time.time()

    ret, img = cap.read()
    rectify = remap(img, map1, map2)
    cv2.imshow(winname, rectify)
#     show = cv2.resize(img, None, fx=0.5, fy=0.5)
#     cv2.imshow('show img', show)
    key = cv2.waitKey(20)
    elps.append((time.time() - start))

    if key == ord('s'):
        cv2.waitKey(300)
        save_imgs.append(img)
    elif key == 27:
        break

    if len(elps) == 100:
        print('- FPS:{}'.format(len(elps) / sum(elps)))
        elps = []

cv2.destroyAllWindows()
print('### finish capturing')
print('#####################')


In [ ]:
Set white balance red: 1.188
    Set white balance blue: 1.5

In [ ]:
1.1800000000000002, 1.46

In [ ]:
2*val/100 + 0.5

In [ ]:
(1.46-0.5)*100/2

In [ ]:
(1.1800000000000002-0.5)*100/2

In [ ]:
plt.imshow(rectify[:,:,::-1])

In [ ]:
plt.imshow(rectify[:,:,::-1])

In [ ]:
plt.imshow(img[:,:,::-1])